# Acquisition, Prep, and Initial Exploration

In [1]:
import pandas as pd
from acquire import acquire_data

In [2]:
df = acquire_data()
df.head()

,parcelid,id,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,...,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,id.1,logerror,transactiondate
0,14297519,1727539,NaN,NaN,NaN,3.5,4.0,NaN,NaN,3.5,...,1023282.0,2016.0,537569.0,11013.72,NaN,NaN,6.059063e+13,0,0.025595,2017-01-01
1,17052889,1387261,NaN,NaN,NaN,1.0,2.0,NaN,NaN,1.0,...,464000.0,2016.0,376000.0,5672.48,NaN,NaN,6.111001e+13,1,0.055619,2017-01-01
2,14186244,11677,NaN,NaN,NaN,2.0,3.0,NaN,NaN,2.0,...,564778.0,2016.0,479489.0,6488.30,NaN,NaN,6.059022e+13,2,0.005383,2017-01-01
3,12177905,2288172,NaN,NaN,NaN,3.0,4.0,NaN,8.0,3.0,...,145143.0,2016.0,36225.0,1777.51,NaN,NaN,6.037300e+13,3,-0.103410,2017-01-01
4,12095076,781532,1.0,NaN,NaN,3.0,4.0,NaN,9.0,3.0,...,773303.0,2016.0,496619.0,9516.26,NaN,NaN,6.037461e+13,6,-0.001011,2017-01-01


## Missing Values

- Handle missing values

- backup/explain your decisions

- Prep.py: Write function for reproducibility.

- Run function in final notebook to complete the task.

In [3]:
def missing_value_percentage(series):
    return series.isna().sum() / series.size
    

In [4]:
def drop_useless_columns(df, percentage):
    #check the percentage of missing value of each column
    s = df.apply(missing_value_percentage, axis = 0)
    # drop columns whose missing values are more than  percentage
    df = df.drop(columns = s[s > percentage].index.tolist()) 
    return df
    

In [5]:
df = drop_useless_columns(df, .3)
df.head()

,parcelid,id,bathroomcnt,bedroomcnt,calculatedbathnbr,calculatedfinishedsquarefeet,finishedsquarefeet12,fips,fullbathcnt,latitude,...,yearbuilt,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,censustractandblock,id.1,logerror,transactiondate
0,14297519,1727539,3.5,4.0,3.5,3100.0,3100.0,6059.0,3.0,33634931.0,...,1998.0,485713.0,1023282.0,2016.0,537569.0,11013.72,6.059063e+13,0,0.025595,2017-01-01
1,17052889,1387261,1.0,2.0,1.0,1465.0,1465.0,6111.0,1.0,34449266.0,...,1967.0,88000.0,464000.0,2016.0,376000.0,5672.48,6.111001e+13,1,0.055619,2017-01-01
2,14186244,11677,2.0,3.0,2.0,1243.0,1243.0,6059.0,2.0,33886168.0,...,1962.0,85289.0,564778.0,2016.0,479489.0,6488.30,6.059022e+13,2,0.005383,2017-01-01
3,12177905,2288172,3.0,4.0,3.0,2376.0,2376.0,6037.0,3.0,34245180.0,...,1970.0,108918.0,145143.0,2016.0,36225.0,1777.51,6.037300e+13,3,-0.103410,2017-01-01
4,12095076,781532,3.0,4.0,3.0,2962.0,2962.0,6037.0,3.0,34145202.0,...,1950.0,276684.0,773303.0,2016.0,496619.0,9516.26,6.037461e+13,6,-0.001011,2017-01-01


In [6]:
def drop_duplicated_observation(df):
    df = df.sort_values('transactiondate', ascending=False)
    #keep the latest transaction date
    return df.drop_duplicates(subset='parcelid', keep='last')

In [7]:
df.shape

(52442, 29)

In [8]:
df = drop_duplicated_observation(df)
df.shape

(52320, 29)

In [9]:
#drop nonuseful columns based on domin knowledge
ineffecitve_columns = ['id','id.1','calculatedbathnbr','finishedsquarefeet12',
                      'fullbathcnt','roomcnt','assessmentyear','censustractandblock',
                      'propertylandusetypeid','rawcensustractandblock','propertycountylandusecode',
                      'transactiondate','parcelid','regionidcounty']
df = df.drop(columns=ineffecitve_columns)
df.head()

,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,fips,latitude,longitude,lotsizesquarefeet,regionidcity,regionidzip,yearbuilt,structuretaxvaluedollarcnt,taxvaluedollarcnt,landtaxvaluedollarcnt,taxamount,logerror
52440,2.0,3.0,1762.0,6037.0,33937685.0,-117996709.0,6347.0,14634.0,96171.0,1955.0,140000.0,522000.0,382000.0,6317.15,0.007204
52439,1.0,3.0,1032.0,6037.0,34040895.0,-118038169.0,5074.0,36502.0,96480.0,1954.0,32797.0,49546.0,16749.0,876.43,0.037129
52438,2.0,4.0,1612.0,6111.0,34300140.0,-118706327.0,12105.0,27110.0,97116.0,1964.0,50683.0,67205.0,16522.0,1107.48,0.013209
52437,2.0,2.0,1286.0,6037.0,34245368.0,-118282383.0,47405.0,12447.0,96284.0,1940.0,70917.0,354621.0,283704.0,4478.43,0.020615
52311,4.0,4.0,2440.0,6037.0,34009367.0,-118430958.0,5553.0,12447.0,96047.0,1937.0,350000.0,1550000.0,1200000.0,18655.00,-0.056152


In [10]:
df.isna().sum()

bathroomcnt                        0
bedroomcnt                         0
calculatedfinishedsquarefeet      81
fips                               0
latitude                           0
longitude                          0
lotsizesquarefeet                366
regionidcity                    1036
regionidzip                       26
yearbuilt                        114
structuretaxvaluedollarcnt        82
taxvaluedollarcnt                  1
landtaxvaluedollarcnt              1
taxamount                          4
logerror                           0
dtype: int64

In [11]:
#drop null observations
df = df.dropna()

## Create new features
- yearbuilt to age
- tax_rate = taxamount / taxvaluedollarcnt
- drop structuretaxvaluedollarcnt, taxvaluedollarcnt,	landtaxvaluedollarcnt,	taxamount

In [12]:
df['age']=(2017 - df.yearbuilt).astype('int')
df['tax_rate']= df.taxamount/df.taxvaluedollarcnt*100
df.head()

,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,fips,latitude,longitude,lotsizesquarefeet,regionidcity,regionidzip,yearbuilt,structuretaxvaluedollarcnt,taxvaluedollarcnt,landtaxvaluedollarcnt,taxamount,logerror,age,tax_rate
52440,2.0,3.0,1762.0,6037.0,33937685.0,-117996709.0,6347.0,14634.0,96171.0,1955.0,140000.0,522000.0,382000.0,6317.15,0.007204,62,1.210182
52439,1.0,3.0,1032.0,6037.0,34040895.0,-118038169.0,5074.0,36502.0,96480.0,1954.0,32797.0,49546.0,16749.0,876.43,0.037129,63,1.768922
52438,2.0,4.0,1612.0,6111.0,34300140.0,-118706327.0,12105.0,27110.0,97116.0,1964.0,50683.0,67205.0,16522.0,1107.48,0.013209,53,1.647913
52437,2.0,2.0,1286.0,6037.0,34245368.0,-118282383.0,47405.0,12447.0,96284.0,1940.0,70917.0,354621.0,283704.0,4478.43,0.020615,77,1.262878
52311,4.0,4.0,2440.0,6037.0,34009367.0,-118430958.0,5553.0,12447.0,96047.0,1937.0,350000.0,1550000.0,1200000.0,18655.00,-0.056152,80,1.203548


In [13]:
df = df.drop(columns=['yearbuilt',
                      'structuretaxvaluedollarcnt', 
                      'taxvaluedollarcnt', 
                      'landtaxvaluedollarcnt',
                      'taxamount'])

In [14]:
df = df.rename(columns={'calculatedfinishedsquarefeet':'house_size', 'lotsizesquarefeet':'lotsize'})

FIPS:

- 6037: Los Angeles County
- 6059: Orange County
- 6111: Ventura County

In [15]:
county_df = pd.get_dummies(df.fips)
county_df.columns = ['LA', 'Orange', 'Ventura']
# concatenate the dataframe with the 3 county columns to the original dataframe
df= pd.concat([df, county_df], axis =1)
# drop regionidcounty and fips columns
df = df.drop(columns = [ 'fips'])
df.head()

,bathroomcnt,bedroomcnt,house_size,latitude,longitude,lotsize,regionidcity,regionidzip,logerror,age,tax_rate,LA,Orange,Ventura
52440,2.0,3.0,1762.0,33937685.0,-117996709.0,6347.0,14634.0,96171.0,0.007204,62,1.210182,1,0,0
52439,1.0,3.0,1032.0,34040895.0,-118038169.0,5074.0,36502.0,96480.0,0.037129,63,1.768922,1,0,0
52438,2.0,4.0,1612.0,34300140.0,-118706327.0,12105.0,27110.0,97116.0,0.013209,53,1.647913,0,0,1
52437,2.0,2.0,1286.0,34245368.0,-118282383.0,47405.0,12447.0,96284.0,0.020615,77,1.262878,1,0,0
52311,4.0,4.0,2440.0,34009367.0,-118430958.0,5553.0,12447.0,96047.0,-0.056152,80,1.203548,1,0,0


## Data Types

- Prep.py: Write a function that takes in a dataframe and a list of column names (ones that are numeric and don't represent numbers) and returns the dataframe with the datatypes of those columns changed to a non-numeric type.

- In your notebook, use this function to appropriately transform any numeric columns that should not be treated as numbers.

- Prep.py: Do the same, but changing objects or categories to numeric types.

In [16]:
df.dtypes

bathroomcnt     float64
bedroomcnt      float64
house_size      float64
latitude        float64
longitude       float64
lotsize         float64
regionidcity    float64
regionidzip     float64
logerror        float64
age               int64
tax_rate        float64
LA                uint8
Orange            uint8
Ventura           uint8
dtype: object

In [19]:
df[['regionidcity','regionidzip']] = df[['regionidcity','regionidzip']].astype('object')
df.dtypes

bathroomcnt     float64
bedroomcnt      float64
house_size      float64
latitude        float64
longitude       float64
lotsize         float64
regionidcity     object
regionidzip      object
logerror        float64
age               int64
tax_rate        float64
LA                uint8
Orange            uint8
Ventura           uint8
dtype: object

## Outliers

- Prep.py: You can use what you did in exercises and adapt, enhance or improve if you find the time and need.

- Prep.py: Write a function that accepts a series (i.e. one column from a data frame) and summarizes how many outliers are in the series. This function should accept a second parameter that determines how outliers are detected, with the ability to detect outliers in 3 ways: IQR, standard deviations (z-score), percentiles).

- Run the function in your final notebook to identify/demonstrat columns where you should handle the outliers.

- Prep.py: Write a function that accepts the zillow data frame and removes the outliers. You should make a decision and document how you will remove outliers.

- Run the function in your final notebook.

- Is there erroneous data you have found that you need to remove or repair? If so, take action.

- Are there outliers you want to "squeeze in" to a max value? (e.g. all bathrooms > 6 => bathrooms = 6). If so, make those changes.

## Other

- Be sure to not forget about the common tasks for this phase of the pipeline (e.g. summarize data, plotting distributions of individual variables).

- Documentation: markdown, docstrings in functions, and comments in code.